<a href="https://colab.research.google.com/github/opassos/FastAI-Exemplos/blob/main/FastAI%20Model%20Splitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
! pip install -Uqqq fastai
! pip install -Uqqq timm

     |████████████████████████████████| 376 kB 5.1 MB/s 


In [29]:
from fastai.vision.all import *
import timm

In [3]:
set_seed(99, True)
path = untar_data(URLs.PETS)/'images'
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2,
    label_func=lambda x: x[0].isupper(), item_tfms=Resize(224))

In [115]:
def uname_model(m):
    l = []
    children = list(m.children())
    for c in children:
        if list(c.children()):
            l.append(uname_model(c))
        else:
            l.append(c)
    return nn.Sequential(*l)

def flatten_model(m):
    l = []
    children = list(m.children())
    for c in children:
        if list(c.children()):
            l.extend(flatten_model(c))
        else:
            l.append(c)
    return nn.Sequential(*l)

def my_split(m): return L(m[:-3], m[-3:]).map(params)

In [116]:
model = resnet18()
learn = Learner(dls, model, splitter = Pipeline([uname_model, my_split]))
learn.freeze()
learn.summary()

ResNet (Input shape: 64)
Layer (type)         Output Shape         Param #    Trainable 
                     64 x 64 x 112 x 112 
Conv2d                                    9408       False     
BatchNorm2d                               128        True      
ReLU                                                           
MaxPool2d                                                      
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
ReLU                                                           
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
ReLU                                                           
Conv2d                                    36864      False     
BatchNorm2d                          

In [117]:
model = resnet18()
learn = Learner(dls, model, splitter = Pipeline([flatten_model, my_split]))
learn.freeze()
learn.summary()

ResNet (Input shape: 64)
Layer (type)         Output Shape         Param #    Trainable 
                     64 x 64 x 112 x 112 
Conv2d                                    9408       False     
BatchNorm2d                               128        True      
ReLU                                                           
MaxPool2d                                                      
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
ReLU                                                           
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
ReLU                                                           
Conv2d                                    36864      False     
BatchNorm2d                          